In [1]:
import multiprocessing
import os
import subprocess
import sys
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from azure.storage.blob import BlobServiceClient
from azure.cli.core import get_default_cli

def az_cli (args_str):
    args = args_str.split()
    cli = get_default_cli()
    cli.invoke(args, out_file = open(os.devnull, 'w'))
    if cli.result.result:
        return cli.result.result
    elif cli.result.error:
        raise cli.result.error
    return True

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# Get Experimentname
if len(sys.argv) == 2:
    experimentname = sys.argv[1]
else:
    experimentname = input("Enter experiment name:")

Enter experiment name:gtbf


In [3]:
azCliSA = az_cli(f"storage account show-connection-string --name {experimentname}core")
download_path = f"./data/downloaded/{experimentname}"
Path(download_path).mkdir(parents=True, exist_ok=True)

In [4]:
blob_service = BlobServiceClient.from_connection_string(azCliSA['connectionString'])
container_client = blob_service.get_container_client("runs")
# List the blobs in the container:
blob_list = container_client.list_blobs()

# List the already downloaded elements:
downloaded_tars = os.listdir(download_path)
downloaded_tars

['20210927T010023935_GCP_us-west2.tar.gz',
 '20210819T133022453_GCP_us-east4.tar.gz',
 '20210806T003043934_AWS_ap-northeast-1.tar.gz',
 '20210913T193022327_GCP_europe-west2.tar.gz',
 '20210907T020023611_GCP_australia-southeast1.tar.gz',
 '20210921T013053359_AWS_sa-east-1.tar.gz',
 '20210802T023050845_AWS_eu-central-1.tar.gz',
 '20210802T160146195_AZURE_germanywestcentral.tar.gz',
 '20210902T170023233_GCP_europe-west2.tar.gz',
 '20210824T020024303_GCP_us-west2.tar.gz',
 '20210818T033055904_AWS_us-west-2.tar.gz',
 '20210818T110027201_AZURE_westus2.tar.gz',
 '20210902T053039528_AWS_us-west-1.tar.gz',
 '20210821T190040106_AZURE_japaneast.tar.gz',
 '20210902T003049510_AWS_ap-south-1.tar.gz',
 '20210924T073023391_GCP_us-west4.tar.gz',
 '20210910T020048024_AWS_eu-central-1.tar.gz',
 '20210903T093023994_GCP_us-west4.tar.gz',
 '20210902T080037130_AZURE_centralindia.tar.gz',
 '20211002T050048844_AWS_us-west-1.tar.gz',
 '20210920T103049578_AWS_us-west-2.tar.gz',
 '20210831T060054934_AWS_ap-southe

In [5]:
def download(blob):
    if blob.name in downloaded_tars:
        return 0

    blob_client = blob_service.get_blob_client(container="runs", blob=blob)
    download_file_path = os.path.join(download_path, blob.name)

    with open(download_file_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())
    return 1

print('downloading')
result = []
with ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()) as tpe:
    result = tpe.map(download, blob_list)

print("downloaded", sum(result))

downloading
downloaded 0
